In [2]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("Data/SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF_IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# Count Vectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
### Exploring parameter settings using GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [7]:
rf = RandomForestClassifier()
param = {'n_estimators': [10,150,300],
        'max_depth':[30, 60 , 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,22.371819,0.596102,0.346202,0.027441,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978456,0.974865,0.975741,0.968553,0.972147,0.973953,0.003368,1
11,38.275737,0.751375,0.310225,0.041920,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.976661,0.974843,0.967655,0.972147,0.973593,0.003398,2
10,23.835783,0.934851,0.342605,0.041724,None,150,"{'max_depth': None, 'n_estimators': 150}",0.976661,0.974865,0.975741,0.966757,0.973046,0.973414,0.003536,3
8,42.824933,0.607171,0.481725,0.018995,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.974865,0.973944,0.967655,0.971249,0.973054,0.003370,4
3,3.109375,0.243093,0.203662,0.015987,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.971275,0.976661,0.976640,0.966757,0.971249,0.972516,0.003755,5


In [8]:
rf = RandomForestClassifier()
param = {'n_estimators': [10,150,300],
        'max_depth':[30, 60 , 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,40.479377,0.512328,0.331618,0.042840,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978456,0.973968,0.975741,0.967655,0.969452,0.973054,0.003984,1
7,23.230135,0.565825,0.371766,0.028612,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.974865,0.975763,0.973944,0.967655,0.971249,0.972695,0.002939,2
10,25.187012,1.185200,0.378741,0.034619,None,150,"{'max_depth': None, 'n_estimators': 150}",0.976661,0.973070,0.973944,0.966757,0.971249,0.972336,0.003291,3
8,45.675903,0.577485,0.555026,0.053652,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.975763,0.971275,0.973046,0.966757,0.970350,0.971438,0.002980,4
6,3.158634,0.073911,0.205757,0.010497,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.968582,0.976661,0.969452,0.967655,0.972147,0.970899,0.003248,5
